# Rough API Run


### Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import math
from PIL import Image
from tensorflow import convert_to_tensor
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
import os
from PIL import ImageFile
from PIL import ImageOps

## The model

In [ ]:
autoencoder_model = load_model('/home/jupyter/artsy-fartsci-back/models/initial_model.h5')

In [ ]:
autoencoder_model.save_weights('/home/jupyter/artsy-fartsci-back/models/initial_weights.hdf5')

## User Input Image

In [ ]:
img_path = os.listdir('/home/jupyter/artsy-fartsci-back/frisbee')

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
input_image_tensor = []

for image_filename in img_path:
    img = Image.open(os.path.join('/home/jupyter/artsy-fartsci-back/frisbee', image_filename))
    img = ImageOps.exif_transpose(img)
    img = img.resize((224,224))
    img_arr = np.array(img)
    img = img_arr/255
    input_image_tensor.append(convert_to_tensor(img))

In [ ]:
frisbee = input_image_tensor[1]

## 500 Dataset Images 

In [ ]:
import os
img_path = os.listdir('/home/jupyter/500/images')

In [ ]:
img_tensors = []

for image_filename in img_path:
    image = Image.open(os.path.join('/home/jupyter/500/images', image_filename))
    image = image.resize((224,224))
    img_tensors.append(convert_to_tensor(image))

In [ ]:
X = np.array(img_tensors)
y = np.array(img_tensors)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

## Autoencoder

In [ ]:
input_img = Input(shape=(224, 224, 3))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
encoder = Model(input_img, encoded)

In [ ]:
autoencoder.load_weights('/home/jupyter/artsy-fartsci-back/weights/weights0039.hdf5')

## Grabbing the five closest

In [ ]:
def get_top_5_similar(input_image, dataset):
    input_representation = encoder.predict(input_image)
    
    # Compute cosine similarities between the input image and all images in the dataset
    similarities = cosine_similarity(input_representation.reshape(1, -1), 
                                     encoder.predict(dataset).reshape(len(dataset), -1))
    
    # Get indices of the top 5 similar images
    top_5_indices = np.argsort(similarities[0])[-5:][::-1]  # -6 because the most similar one will be the image itself
    
    return top_5_indices

# Example of getting top 5 similar images to x_test[111]

input_image = np.expand_dims(frisbee, axis = 0)
similar_indices = get_top_5_similar(input_image, X_test)
print(f"Indices of top 5 similar images: {similar_indices}")

# Display the input image and its 5 most similar images
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 6, figsize=(15, 5))
ax[0].imshow(input_image[0])
ax[0].set_title("Input Image")
ax[0].axis('off')

for i, index in enumerate(similar_indices, 1):
    ax[i].imshow(X_test[index])
    ax[i].set_title(f"Similar {i}")
    ax[i].axis('off')

plt.show()